![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [5]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

rental_data = pd.read_csv('rental_info.csv')



In [6]:

rental_data['rental_date'] = pd.to_datetime(rental_data['rental_date'])
rental_data['return_date'] = pd.to_datetime(rental_data['return_date'])

rental_data['rental_length_days'] = (rental_data['return_date'] - rental_data['rental_date']).dt.days



In [7]:
rental_data['deleted_scenes'] = rental_data['special_features'].apply(lambda x: 1 if 'Deleted Scenes' in str(x) else 0)
rental_data['behind_the_scenes'] = rental_data['special_features'].apply(lambda x: 1 if 'Behind the Scenes' in str(x) else 0)


features = [
    'amount', 'amount_2', 'rental_rate', 'rental_rate_2', 'release_year',
    'length', 'length_2', 'replacement_cost', 'deleted_scenes', 'behind_the_scenes',
    'NC-17', 'PG', 'PG-13', 'R'
]
X = rental_data[features]
y = rental_data['rental_length_days']



In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)


linear_model = LinearRegression()
decision_tree = DecisionTreeRegressor(random_state=9)
random_forest = RandomForestRegressor(random_state=9)

models = [(linear_model, "Linear Regression"), (decision_tree, "Decision Tree"), (random_forest, "Random Forest")]
mse_scores = {}

for model, name in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_scores[name] = mse
    print(f"{name} MSE: {mse}")

best_model_name = min(mse_scores, key=mse_scores.get)
best_model = [model for model, name in models if name == best_model_name][0]
best_mse = mse_scores[best_model_name]

print(f"\nRecommended Model: {best_model_name}")
print(f"Best MSE: {best_mse}")


Linear Regression MSE: 2.9417238646976065
Decision Tree MSE: 2.1584260730200473
Random Forest MSE: 2.0302919867818305

Recommended Model: Random Forest
Best MSE: 2.0302919867818305
